In [5]:
import pandas as pd
import networkx as nx
from collections import defaultdict

In [6]:
titanic_df = pd.read_parquet('data/titanic_data.parquet.gzip')

In [7]:
def remove_lonely(name):
    if name in lonely:
        return 0
    else:
        return 1

In [8]:
titanic_df['surname'] = titanic_df.Name.apply(lambda x: x.split(',')[0])
surnames = titanic_df['surname'].value_counts()
lonely = []
for index, item in surnames.iteritems():
    if item == 1:
        lonely.append(index)
titanic_df = titanic_df[titanic_df['surname'].apply(remove_lonely) != 0]

In [9]:
titanic_passengers = dict()
for passenger in titanic_df.index:
    titanic_passengers[titanic_df.loc[passenger, 'PassengerId']] = {
        'Survived': titanic_df.loc[passenger, 'Survived'],
        'Pclass': titanic_df.loc[passenger, 'Pclass'],
        'Name': titanic_df.loc[passenger, 'Name'],
        'Sex': titanic_df.loc[passenger, 'Sex'],
        'Age': titanic_df.loc[passenger, 'Age'],
        'SibSp': titanic_df.loc[passenger, 'SibSp'],
        'Parch': titanic_df.loc[passenger, 'Parch'],
        'surname': titanic_df.loc[passenger, 'surname'],
    }

In [10]:
graph = defaultdict(list)
for passenger_id in titanic_df.PassengerId.tolist():
    surname = titanic_df[titanic_df['PassengerId'] == passenger_id].surname
    surname = surname.values[0]
    family = titanic_df[titanic_df['surname'] == surname]
    for member in family.PassengerId.tolist():
        if member != passenger_id:
            graph[passenger_id].append(member)

In [45]:
titanic_df['weight'] = titanic_df.apply(lambda x: x.Age + 35 if x.Sex == 'male' else x.Age + 25, axis=1)